In [41]:
from dataclasses import dataclass, field
from enum import Enum, auto
from typing import List, Union, Dict, Tuple, Optional, Any, TypeAlias, Iterable, Callable
import re

#Parameters
filepath = "Caves/easy/path_e1.txt"

# <editor-fold desc="Semantics and structure for FOL">

class TokenType(Enum):
    XOR = auto()
    OR = auto()
    AND = auto()
    NOT = auto()
    ALL = auto()
    ANY = auto()
    IMPLIES = auto()
    IFF = auto()
    IN = auto()

    TRUE = auto()
    FALSE = auto()
    IDENT = auto()

    LPAREN = auto()
    RPAREN = auto()
    COMMA  = auto()

    def __str__(self): return self.name


@dataclass(frozen=True)
class Token:
    type: TokenType
    name: Optional[str] = None  # used for variables quantifiers and predicates

class Lexer:
    _whitespace = re.compile(r"\s+")
    _identifier = re.compile(r"[A-Za-z_][A-Za-z0-9_]*")

    SYMBOLS = [
        ("(", TokenType.LPAREN),
        (")", TokenType.RPAREN),
        (",", TokenType.COMMA),
    ]

    # keywords, case-sensitive
    KEYWORDS = {
        "TRUE": TokenType.TRUE,
        "FALSE": TokenType.FALSE,

        "AND": TokenType.AND,
        "OR": TokenType.OR,
        "XOR": TokenType.XOR,
        "NOT": TokenType.NOT,

        "IMPLIES": TokenType.IMPLIES,
        "IFF": TokenType.IFF,

        "ALL": TokenType.ALL,
        "ANY": TokenType.ANY,
        "IN": TokenType.IN,
    }

    def tokenize(self, text: str) -> List[Token]:
        tokens: List[Token] = []
        i = 0
        length = len(text)

        while i < length:
            # skip whitespace
            current = self._whitespace.match(text, i)
            if current:
                i = current.end()
                if i >= length:
                    break

            matched_symbol = False
            for symbol, token_type in self.SYMBOLS:
                if text.startswith(symbol, i):
                    tokens.append(Token(token_type))
                    i += len(symbol)
                    matched_symbol = True
                    break
            if matched_symbol:
                continue

            # Identifier / keyword
            current = self._identifier.match(text, i)
            if current:
                lex = current.group(0)
                i = current.end()

                # Keyword
                token_type = self.KEYWORDS.get(lex)
                if token_type is not None:
                    tokens.append(Token(token_type))
                else:
                    tokens.append(Token(TokenType.IDENT, name=lex))
                continue

            # fallback
            raise ValueError(f"Unexpected character at {i}: {repr(text[i])}")

        return tokens

# Terms
class LogicTerminal(Enum):
    U = auto()  # Unknown
    F = auto()  # False
    T = auto()  # True

    def __str__(self):
        if self is LogicTerminal.U: return "Unknown"
        if self is LogicTerminal.F: return "False"
        if self is LogicTerminal.T: return "True"
        return "NULL"

    def __bool__(self): return self is LogicTerminal.T


@dataclass(frozen=True)
class Variable:
    name: Any
    def __str__(self) -> str: return str(self.name)

@dataclass(frozen=True)
class Constant:
    value: LogicTerminal = LogicTerminal.U
    def __str__(self) -> str: return str(self.value)

@dataclass(frozen=True)
class Predicate:
    name: str
    args: Tuple[Any, ...] = field(default_factory=tuple)  # must be immutable for hashing

    def __str__(self):
        return f"{self.name}({', '.join(map(str, self.args))})"


class LogicOperator(Enum):
    XOR = auto()
    AND = auto()
    OR = auto()
    IFF = auto()
    IMPLIES = auto()

    def __str__(self): return self.name

class Quantifier(Enum):
    ANY = auto()
    ALL = auto()

@dataclass(frozen=True)
class Not:
    child: Any

@dataclass(frozen=True)
class Operator:
    nodeType: LogicOperator
    children: List[Any] = field(default_factory=list)

@dataclass(frozen=True)
class QuantifierExpression:
    quantifier: Quantifier
    variables: Tuple[Variable, ...]
    domain: Any
    expression: 'Expression'

Term: TypeAlias = Union[Variable, Constant, LogicTerminal, None]
Expression: TypeAlias = Union[Predicate, Not, Operator, QuantifierExpression, Constant, Variable]
ParseNode = Union[Term, Expression]



class Parser:
    def __init__(self):
        self.expression: List[Token] = []
        self.parse_index = 0

    def __call__(self, arg: Union[str,List[Token]]):
        if isinstance(arg, str):
            lex = Lexer()
            self.expression = lex.tokenize(arg)
        elif isinstance(arg, list):
            self.expression = arg
        return self.parse(self.expression)

    def parse(self, tokens: List[Token]) -> ParseNode:
        self.expression = tokens
        self.parse_index = 0
        return self.parse_expression()

    def peek(self, k=0) -> Optional[Token]:
        i = self.parse_index + k
        return self.expression[i] if 0 <= i < len(self.expression) else None

    def peek_is(self, t: TokenType) -> bool:
        """
        Checks if next token exists and is of a certain type
        """
        tok = self.peek()
        return tok is not None and tok.type is t

    def eat(self) -> Optional[Token]:
        tok = self.peek()
        if tok is not None:
            self.parse_index += 1
        return tok

    def expect(self, token_type: TokenType) -> Token:
        tok = self.eat()
        if tok is None or tok.type is not token_type:
            raise ValueError(f"Expected {token_type}, got {tok}")
        return tok

    def parse_expression(self) -> ParseNode:
        node = self._parse_iff()
        if self.peek() is not None:
            raise ValueError(f"Expression not empty after parsing")
        return node

    def _parse_iff(self) -> ParseNode:
        node = self._parse_implies()
        while self.peek_is(TokenType.IFF):
            self.eat()
            rhs = self._parse_implies()
            node = self._reduce_iff(node, rhs)
        return node

    def _parse_implies(self) -> ParseNode:
        left = self._parse_xor()
        if self.peek_is(TokenType.IMPLIES):
            self.eat()
            right = self._parse_implies()  # right-assoc
            return self.reduce_implies(left, right)
        return left

    def _parse_xor(self) -> ParseNode:
        node = self._parse_or()
        while self.peek_is(TokenType.XOR):
            self.eat()
            rhs = self._parse_or()
            node = Operator(nodeType=LogicOperator.XOR, children=[node, rhs])
        return node

    def _parse_or(self) -> ParseNode:
        node = self._parse_and()
        while self.peek_is(TokenType.OR):
            self.eat()
            rhs = self._parse_and()
            node = Operator(nodeType=LogicOperator.OR, children=[node, rhs])
        return node

    def _parse_and(self) -> ParseNode:
        node = self._parse_not()
        while self.peek_is(TokenType.AND):
            self.eat()
            rhs = self._parse_not()
            node = Operator(nodeType=LogicOperator.AND, children=[node, rhs])
        return node

    def _parse_not(self) -> ParseNode:
        if self.peek_is(TokenType.NOT):
            self.eat()
            return Not(child=self._parse_not())
        return self._parse_atom()

    def _parse_atom(self) -> ParseNode:
        tok = self.peek()
        if tok is None:
            raise ValueError("Unexpected end of expression")

        # booleans
        if tok.type is TokenType.TRUE:
            self.eat()
            return Constant(LogicTerminal.T)
        if tok.type is TokenType.FALSE:
            self.eat()
            return Constant(LogicTerminal.F)

        #'(' expr ')'
        if tok.type is TokenType.LPAREN:
            self.eat()
            node = self._parse_iff()
            self.expect(TokenType.RPAREN)
            return node

        # Quantifier (ALL/ANY)
        if self.peek_is(TokenType.ALL) or self.peek_is(TokenType.ANY):
            return self._parse_quantifier()

        # Predicate or variable: IDENT [ '(' args ')' ]
        if tok.type is TokenType.IDENT:
            ident = self.eat()
            name = ident.name

            if self.peek_is(TokenType.LPAREN):
                self.eat()
                args: List[Term] = []
                if not self.peek_is(TokenType.RPAREN):
                    while True:
                        args.append(self._parse_term())
                        if self.peek_is(TokenType.COMMA):
                            self.eat()
                            continue
                        break
                self.expect(TokenType.RPAREN)
                return Predicate(name=name, args=tuple(args))

            # variable
            return Variable(name=name)

        raise ValueError(f"Unexpected token in atom: {tok}")

    def _parse_term(self) -> Term:
        tok = self.peek()
        if tok is None:
            raise ValueError("Unexpected end of arguments")
        if tok.type is TokenType.TRUE:
            self.eat()
            return Constant(LogicTerminal.T)
        if tok.type is TokenType.FALSE:
            self.eat()
            return Constant(LogicTerminal.F)
        if tok.type is TokenType.IDENT:
            return Variable(name=self.eat().name)
        if tok.type is TokenType.LPAREN:
            raise ValueError("Nested predicate arguments not supported")
        raise ValueError(f"Invalid token: {tok}")

    def _parse_quantifier(self) -> ParseNode:
        quantifier_token = self.eat()
        quantifier = Quantifier.ALL if quantifier_token.type is TokenType.ALL else Quantifier.ANY

        # Variables: IDENT or '(' IDENT (',' IDENT)* ')'
        variables: List[Variable] = []
        if self.peek_is(TokenType.LPAREN):
            self.eat()
            while True:
                ident = self.expect(TokenType.IDENT)
                variables.append(Variable(name=ident.name))
                if self.peek_is(TokenType.COMMA):
                    self.eat()
                    continue
                break
            self.expect(TokenType.RPAREN)
        else:
            ident = self.expect(TokenType.IDENT)
            variables.append(Variable(name=ident.name))

        # IN domain
        self.expect(TokenType.IN)
        domain_token = self.expect(TokenType.IDENT)
        domain = domain_token.name

        if self.peek_is(TokenType.LPAREN):
            self.eat()
            body = self._parse_iff()
            self.expect(TokenType.RPAREN)
        else:
            body = self._parse_iff()

        return QuantifierExpression(quantifier=quantifier, variables=tuple(variables), domain=domain, expression=body)

    # Reductions for IMPLIES/IFF
    @staticmethod
    def reduce_implies(p: ParseNode, q: ParseNode) -> ParseNode:
        # p -> q  ==  (!p) OR q
        return Operator(nodeType=LogicOperator.OR, children=[Not(p), q])

    def _reduce_iff(self, p: ParseNode, q: ParseNode) -> ParseNode:
        # p <-> q  ==  (p -> q) AND (q -> p)
        return Operator(
            nodeType=LogicOperator.AND,
            children=[self.reduce_implies(p, q), self.reduce_implies(q, p)]
        )

    @staticmethod
    def pretty_print(node: ParseNode, indent: str = "", is_last: bool = True):
        branch = "\\-- " if is_last else "| "
        next_indent = indent + ("    " if is_last else "|   ")

        # Operator
        if isinstance(node, Operator):
            label = str(node.nodeType.name)
            print(indent + branch + label)
            for i, child in enumerate(node.children):
                Parser.pretty_print(child, next_indent, i == len(node.children) - 1)
            return

        # NOT
        if isinstance(node, Not):
            print(indent + branch + "NOT")
            Parser.pretty_print(node.child, next_indent, True)
            return

        # Predicate
        if isinstance(node, Predicate):
            print(indent + branch + str(node))
            return

        # Constant (LogicTerminal)
        if isinstance(node, Constant):
            print(indent + branch + str(node.value))
            return

        # Variable
        if isinstance(node, Variable):
            print(indent + branch + f"Variable({node.name})")
            return

        # Quantifier
        if isinstance(node, QuantifierExpression):
            quantifier_name = str(node.quantifier.name)
            variables_str = ", ".join(v.name for v in node.variables)
            domain_str = node.domain
            label = f"{quantifier_name} {variables_str} IN {domain_str}"
            print(indent + branch + label)
            Parser.pretty_print(node.expression, next_indent, True)
            return

        # LogicTerminal
        if isinstance(node, LogicTerminal):
            print(indent + branch + str(node))
            return

        # Fallback
        print(indent + branch + f"{node}")



class ExpressionEvaluator:
    """
    Evaluate an AST
    - For propositional variables: look up in variable_environment (by variable name)
    - For predicates consult predicate_table
    - For quantifiers need domains
    """

    def __init__(
        self,
        root: ParseNode,
        variable_environment: Optional[Dict[str, LogicTerminal]] = None,
        predicate_table: Optional[Dict[Tuple[str, Tuple[Any, ...]], LogicTerminal]] = None,
        domains: Optional[Dict[str, Union[Iterable,Callable]]] = None,
    ):
        self.root = root
        self.variable_environment = variable_environment or {}
        self.predicate_table = predicate_table or {}
        self.domains = domains or {}

        # current variable bindings
        self.bindings: Dict[str, Any] = {}

        self.evaluation: LogicTerminal = self.eval(root)

    def _resolve_domain(self, name: str) -> Iterable[Any]:
        if name not in self.domains:
            raise ValueError(f"Domain '{name}' not provided.")
        domain = self.domains[name]
        # If callable, pass current bindings so it can depend on bound vars
        return domain(self.bindings) if callable(domain) else domain

    @staticmethod
    def implies(p: ParseNode, q: ParseNode) -> ParseNode:
        return Operator(LogicOperator.OR, [Not(p), q])

    @staticmethod
    def equivalence(p: ParseNode, q: ParseNode) -> LogicTerminal:
        """
        Evaluate (p -> q) ∧ (q -> p).
        """
        new_expression_tree = Operator(LogicOperator.AND, [ExpressionEvaluator.implies(p, q), ExpressionEvaluator.implies(q, p)])
        return ExpressionEvaluator(new_expression_tree).evaluation

    @staticmethod
    def _not(arg: LogicTerminal) -> LogicTerminal:
        match arg:
            case LogicTerminal.F: return LogicTerminal.T
            case LogicTerminal.T: return LogicTerminal.F
            case _: return LogicTerminal.U

    @staticmethod
    def _and(args: List[LogicTerminal]) -> LogicTerminal:
        """
        True IFF all entries are True. If at least one is false, false, if at least one is unknown, unknown
        """
        if any(arg is LogicTerminal.F for arg in args): return LogicTerminal.F
        if any(arg is LogicTerminal.U for arg in args): return LogicTerminal.U
        return LogicTerminal.T

    @staticmethod
    def _or(args: List[LogicTerminal]) -> LogicTerminal:
        """
        True IFF any entries are True. Then Unknown if any are unknown, false otherwise
        """
        if any(arg is LogicTerminal.T for arg in args): return LogicTerminal.T
        if any(arg is LogicTerminal.U for arg in args): return LogicTerminal.U
        return LogicTerminal.F

    @staticmethod
    def _xor(args: List[LogicTerminal]) -> LogicTerminal:
        """
        True if only one entry is true, with no unknown entries.
        If the number of true args is <=1 then the evaluation can be unknown if unknown is present.
        Otherwise false
        """
        true_count = sum(int(bool(arg)) for arg in args)
        if true_count > 1:
            return LogicTerminal.F
        if LogicTerminal.U in args:
            return LogicTerminal.U
        return LogicTerminal.T if true_count == 1 else LogicTerminal.F



    def eval(self, node: ParseNode) -> LogicTerminal:
        # Constants / raw terminals
        if isinstance(node, Constant): return node.value
        if isinstance(node, LogicTerminal): return node

        # Propositional variable alone
        if isinstance(node, Variable):
            return self._eval_variable(node)

        # Predicates (possibly with vars)
        if isinstance(node, Predicate):
            return self._eval_predicate(node)

        # Unary NOT
        if isinstance(node, Not):
            return self._not(self.eval(node.child))

        # Operators (AND/OR/XOR/IMPLIES/IFF)
        if isinstance(node, Operator):
            op = node.nodeType
            vals = [self.eval(c) for c in node.children]
            if op is LogicOperator.AND:     return self._and(vals)
            if op is LogicOperator.OR:      return self._or(vals)
            if op is LogicOperator.XOR:     return self._xor(vals)
            if op is LogicOperator.IMPLIES: # reduce here just in case
                if len(vals) != 2:
                    return LogicTerminal.U
                return self._or([self._not(vals[0]), vals[1]])
            if op is LogicOperator.IFF:
                if len(vals) != 2:
                    return LogicTerminal.U
                # (a↔b) := (a->b) ∧ (b->a)
                ab = self._or([self._not(vals[0]), vals[1]])
                ba = self._or([self._not(vals[1]), vals[0]])
                return self._and([ab, ba])
            raise ValueError(f"Unsupported operator: {op}")

        # Quantifiers
        if isinstance(node, QuantifierExpression):
            return self._eval_quantifier(node)

        # Fallback
        raise ValueError(f"Cannot evaluate node of type {type(node).__name__}")

    def _eval_variable(self, variable: Variable) -> LogicTerminal:
        if variable.name in self.bindings:
            bound = self.bindings[variable.name]
            if isinstance(bound, bool):
                return LogicTerminal.T if bound else LogicTerminal.F
            if isinstance(bound, LogicTerminal):
                return bound
            return LogicTerminal.U
        # Propositional variable lookup by name
        return self.variable_environment.get(str(variable.name), LogicTerminal.U)

    def _eval_predicate(self, predicate: Predicate) -> LogicTerminal:
        # replace Variable by their bound values if present.
        predicate_args: Tuple[Any, ...] = tuple(self.bindings.get(arg.name, arg) if isinstance(arg, Variable) else arg for arg in predicate.args)

        # lookup in predicate table
        key = (predicate.name, predicate_args)
        if key in self.predicate_table:
            return self.predicate_table[key]

        # unknown if lookups do not yield truth value
        return LogicTerminal.U

    def _eval_quantifier(self, quantifier: QuantifierExpression) -> LogicTerminal:
        # get domain
        if isinstance(quantifier.domain, str):
            domain_iterable = self._resolve_domain(quantifier.domain)
        else:
            domain_iterable = quantifier.domain

        variables = list(quantifier.variables)
        if not variables:
            return self.eval(quantifier.expression)

        # tries combinations of facts to find truth value
        def assign_and_eval(arg_index: int) -> LogicTerminal:
            if arg_index == len(variables):
                return self.eval(quantifier.expression)  # all variables bound
            variable = variables[arg_index]
            result_accumulator: List[LogicTerminal] = []
            for element in domain_iterable:
                self.bindings[variable.name] = element
                value = assign_and_eval(arg_index + 1) # Recurse and assign next value
                result_accumulator.append(value)

                if quantifier.quantifier is Quantifier.ALL and value is LogicTerminal.F:
                    del self.bindings[variable.name]
                    return LogicTerminal.F
                if quantifier.quantifier is Quantifier.ANY and value is LogicTerminal.T:
                    del self.bindings[variable.name]
                    return LogicTerminal.T

            # Clean binding for this variable
            if variable.name in self.bindings:
                del self.bindings[variable.name]

            # Aggregate Unknowns
            if quantifier.quantifier is Quantifier.ALL:
                return self._and(result_accumulator)
            else:
                return self._or(result_accumulator)

        return assign_and_eval(0)


# </editor-fold>

# <editor-fold desc="Wumpis World">

class Safety(Enum):
    SAFE = auto()
    RISKY = auto()
    UNSAFE = auto()
    UNKNOWN = auto()

class PuzzleParser:
    def __init__(self):
        self.size: Tuple[int,int] = (-1, -1)
        self.arrows: int = -1
        self.path:Dict[Tuple:Dict[str:bool]] = {} # Relates Position to boolean values of Breeze and Stench
        self.query:Tuple = (-1,-1)
        self.resolution: Safety = Safety.UNKNOWN
        self.file_read = False

        try:
            self.parse_puzzle()
            self.file_read = True
        except FileNotFoundError:
            print(f"File {filepath} not found")
            self.file_read = False
        except Exception:
            print(f"Bad File: {filepath}")
            self.file_read = False

    def __bool__(self):
        return self.file_read

    def parse_puzzle(self):
        with open(filepath) as file:
            path: List[str] = []
            for raw in file.readlines():
                line = raw.strip()
                if line.startswith('GRID: '):
                    grid = line.replace('GRID: ', '')
                    self.size = tuple(map(int, grid.split('x')))
                if line.startswith('ARROWS: '):
                    self.arrows = int(line.replace('ARROWS: ', ''))
                if line.startswith('QUERY: '):
                    query = line.replace('QUERY: (', '')[:-1]
                    self.query = tuple(map(int, query.split(',')))
                if line.startswith('RESOLUTION: '):
                    self.resolution = Safety[line.replace('RESOLUTION: ', '')]
                if line.startswith('('):
                    path.append(line)

            for step in path:
                position, breeze, stench = tuple(step[:-1].split(' '))
                position = position[1:-1]
                row, col = tuple(map(int, position.split(',')))
                breeze = breeze[-1] == 'T'
                stench = stench[-1] == 'T'
                self.path[(row, col)] = {"Breeze": breeze, "Stench": stench}

    def get_size(self):
        return self.size

    def get_path(self):
        return self.path



class KnowledgeBase:
    def __init__(self):
        # lowercase tuple in these definitions is a coordinate pair
        self.rules: Optional[List[ParseNode]] = []
        self.facts: Optional[Dict[Tuple[str,tuple]:LogicTerminal]] = {}
        self.domains: Optional[Dict[str,List[tuple]]] = {}

        self.puzzle = PuzzleParser()
        if not self.puzzle: return

        self.domains = {
            'Cells':[]
        }
        for row in range(self.puzzle.size[0]):
            for col in range(self.puzzle.size[1]):
                self.domains['Cells'].append((row, col))

        self.logic_parser = Parser()
        self.rules.extend(map(self.logic_parser,[
            "ALL x IN Cells ( Wumpis(x) IMPLIES (NOT Safe(x)) )",
            "ALL x IN Cells ( Pit(x) IMPLIES (NOT Safe(x)) )",
            "ALL x IN Cells ( ((NOT Pit(x)) AND (NOT Wumpis(x))) IMPLIES Safe(x) )",
        ]))


    def get_neighbors(self,square: tuple)->List[Tuple[int,int]]:
        neighbors: List[Tuple[int,int]] = []
        xbounds,ybounds = zip((0,0),self.puzzle.get_size())

        for diff in [(1,0),(-1,0),(0,1),(0,-1)]:
            neighbor = (square[0] + diff[0], square[1] + diff[1])
            if neighbor[0] < xbounds[0] or neighbor[0] >= xbounds[1]:
                continue
            if neighbor[1] < ybounds[0] or neighbor[1] >= ybounds[1]:
                continue
            neighbors.append(neighbor)

        return neighbors

    def get_puzzle_facts(self):
        path = self.puzzle.get_path()
        for key in path:
            self.facts[("Breeze",tuple(key))] = LogicTerminal.T if path[key]["Breeze"] else LogicTerminal.F
            self.facts[("Stench",tuple(key))] = LogicTerminal.T if path[key]["Stench"] else LogicTerminal.F

        for (name, (row, col)), value in self.facts.items():
            if name == "Stench" and value == LogicTerminal.T:
                self.rules.append(self.logic_parser(f"ANY x IN neighbor{row}_{col} ( Wumpis(x))"))
                self.domains[f"neighbor{row}_{col}"]=self.get_neighbors((row,col))
            elif name == "Breeze" and value == LogicTerminal.T:
                self.rules.append(self.logic_parser(f"ANY x IN neighbor{row}_{col} ( Pit(x)))"))
                self.domains[f"neighbor{row}_{col}"]=self.get_neighbors((row,col))



class InferenceEngine:
    # TODO: Write this class
    def __init__(self):
        self.knowledge = KnowledgeBase()

    def get_knowledge_base(self):
        return self.knowledge

    def unify(self, a: Any, b: Any, theta: Optional[Dict[Any, Any]] = None) -> Optional[Dict[Any, Any]]:
        """
        Try to unify two terms/literals/predicates.
        Return a substitution dict if successful, else None.
        """
        # TODO: implement Robinson unification (occurs check, recursive structure)
        return None

    def derive(self, max_iterations: int = 1000) -> int:
        """
        Saturate the KB by forward-chaining: repeatedly apply rules to add new facts.
        Returns number of new facts derived.
        """
        # TODO: forward chaining / resolution
        new_count = 0
        return new_count

    def classify(self, cell: Tuple[int, int]) -> Safety:
        """
        Use the KB to classify a cell as SAFE/UNSAFE/RISKY/UNKNOWN.
        """
        # TODO: encode domain-specific predicates, e.g., SAFE(r,c), PIT(r,c), WUMPUS(r,c)
        return Safety.UNKNOWN

class OutputWriter:
    # TODO: Write this class
    def __init__(self):
        self.engine = InferenceEngine()
        self.knowledge = self.engine.get_knowledge_base()

    def write_result(self, kb: KnowledgeBase) -> None:
        """
        Emit a final report describing how the puzzle was solved:
        - metrics
        - facts used
        - (optionally) rules and/or proof traces
        """
        pass

# </editor-fold>

lex = Lexer()
parser = Parser()
ast = parser.parse(lex.tokenize("ALL x IN D ( P(x) IMPLIES ( Q(x) AND NOT K(x) ) )"))
Parser.pretty_print(ast)

domains = {"D": [1,2]}
preds = {
    ("P", (1,)): LogicTerminal.T,
    ("P", (2,)): LogicTerminal.F,
    ("Q", (1,)): LogicTerminal.F,
    ("Q", (2,)): LogicTerminal.T,
}
print(ExpressionEvaluator(ast, predicate_table=preds, domains=domains).evaluation)  # -> True
KnowledgeBase()


\-- ALL x IN D
    \-- OR
        | NOT
        |   \-- P(x)
        \-- AND
            | Q(x)
            \-- NOT
                \-- K(x)
False
